### Model Training

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from catboost import CatBoostRegressor
import warnings

In [30]:
# importing data
df=pd.read_csv('./stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [31]:
X=df.drop(columns=['math_score'],axis=1)
y=df['math_score']

In [32]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [33]:
print("categories in gender variable: ",end=" ")
print(df['gender'].unique())
print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

categories in gender variable:  ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [34]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [35]:
# Create Column Transformer with 3 types of transformers

cat_features=X.select_dtypes(include='object').columns
numeric_features=X.select_dtypes(exclude='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()
preprocessor=ColumnTransformer(
    [
        ("OneHotEncoding",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,numeric_features)
    ]
)

In [36]:
X=preprocessor.fit_transform(X)

In [37]:
X.shape

(1000, 19)

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=41)
X_train.shape,X_test.shape

((800, 19), (200, 19))

In [39]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [40]:
models={
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-neighbors Regressor":KNeighborsRegressor(),
    "Decision Tree Regressor":DecisionTreeRegressor(),
    "svm":SVR(),
    "Random forest":RandomForestRegressor(),
    "Xgboost":XGBRegressor(),
    "catBoost":CatBoostRegressor(),
    "Adaboost":AdaBoostRegressor()
}
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)   #train model

    ## predict model
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    ## evalucation
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3151
- Mean Absolute Error: 4.2463
- R2 Score: 0.8808
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3851
- Mean Absolute Error: 4.2608
- R2 Score: 0.8553


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5752
- Mean Absolute Error: 5.1826
- R2 Score: 0.8176
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.3554
- Mean Absolute Error: 5.0539
- R2 Score: 0.7984


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3154
- Mean Absolute Error: 4.2455
- R2 Score: 0.8808
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3801
- Mean Absolute Error: 4.2573
- R2 Score: 0.8556


K-neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.8232
- Mean Absolute Error: 4.6010
- R2 Score: 0.8569
-----------------------

# Result

In [43]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name',"R2_Score"]).sort_values(by=['R2_Score'],ascending=False)

,Model Name,R2_Score
2,Ridge,0.855563
0,Linear Regression,0.855295
8,catBoost,0.828297
6,Random forest,0.815186
7,Xgboost,0.800727
1,Lasso,0.798450
5,svm,0.794986
9,Adaboost,0.785864
3,K-neighbors Regressor,0.759405
4,Decision Tree Regressor,0.624806
